In [205]:
import pandas as pd
import numpy as np

In [244]:
covid_owid = pd.read_csv("../data_source/owid-covid-data.csv")
countries = pd.read_csv("../data_source/countries.csv")

In [245]:
covid_owid.shape

(157936, 67)

In [246]:
countries.shape

(48, 39)

## Covid Treatment

Our target is work with Europe, first let's check all the continents stored and which percetange they represent 

In [247]:
continent_counts = covid_owid.loc[:, ["location","continent"] ].groupby(["continent"]).count()
continent_counts["percentage"] = continent_counts["location"] / covid_owid.shape[0]
continent_counts.sort_values("percentage", ascending=False)

,location,percentage
continent,,
Africa,37437,0.237039
Europe,35256,0.223230
Asia,34445,0.218095
North America,23669,0.149865
South America,9114,0.057707
Oceania,8527,0.053990


As we can observe our target represents 22% of the population. Now let's filter our full dataset by only our case and some columns selected 

In [248]:
columns_to_be_analyzed = [
    "location"
    , "date"
    , "total_cases"
    , "new_cases"
    , "total_deaths"
    , "new_deaths"
    , "icu_patients"
    , "hosp_patients"
    , "total_tests"
    , "positive_rate"
    , "total_vaccinations"
    , "people_vaccinated"
    , "people_fully_vaccinated"
    , "life_expectancy"
]

In [249]:
covid_europe = covid_owid.loc[ covid_owid["continent"] == "Europe" , columns_to_be_analyzed ]

For country let's analyze percentage of NaNs by column:

In [250]:
nans_columns = covid_europe.groupby(["location"], as_index=False).count()

In [251]:
for c_ in  nans_columns.columns:
    if c_ not in ['location','date']:
        nans_columns[c_] = 1 - nans_columns[c_] / nans_columns["date"] 

Based on our current variables let's analyze which of them can have NaNs:

- total_cases: that one should be have always values. (IMPORTANT)
- new_cases: we can have NaNs cases, not all days have new cases. > .9 nans will be excluded 
- total_deaths: that one should be have always values. (IMPORTANT)
- new_deaths: we can have NaNs cases, not all days have new cases.
- icu_patients:we can have NaNs cases, not all days have new cases.
- hosp_patients:we can have NaNs cases, not all days have new cases. > .9 nans will be excluded
- total_tests: we can have NaNs cases, not all days have new cases. > .9 nans will be excluded
- positive_rate: we can have NaNs cases, not all days have new cases. > .9 nans will be excluded
- total_vaccinations: we can have NaNs cases, not all days have new cases. 1.0 nans will be excluded
- people_vaccinated: we can have NaNs cases, not all days have new cases. .9 nans will be excluded
- people_fully_vaccinated: we can have NaNs cases, not all days have new cases.
- life_expectancy: that one should be have always values. (IMPORTANT)


In [252]:
nans_columns.shape

(51, 14)

In [253]:
nans_columns = nans_columns.loc[
    (nans_columns["total_cases"] < 1.0)
    & (nans_columns["new_cases"] < 0.9 )
    & (nans_columns["total_deaths"] < 1.0 )
    & (nans_columns["hosp_patients"] < 0.9 )
    & (nans_columns["total_tests"] < 0.9 )
    & (nans_columns["positive_rate"] < 0.9 )
    & (nans_columns["total_vaccinations"] < 1.0 )
    & (nans_columns["people_vaccinated"] < 0.9 )
    & (nans_columns["life_expectancy"] < 1.0 )
    ,
    :
]

In [254]:
nans_columns.shape

(24, 14)

In [255]:
covid_europe = covid_europe.loc[ covid_europe["location"].isin( nans_columns["location"].array ) , : ] 

In [256]:
covid_europe

,location,date,total_cases,new_cases,total_deaths,new_deaths,icu_patients,hosp_patients,total_tests,positive_rate,total_vaccinations,people_vaccinated,people_fully_vaccinated,life_expectancy
9183,Austria,2020-02-25,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.54
9184,Austria,2020-02-26,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.54
9185,Austria,2020-02-27,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.54
9186,Austria,2020-02-28,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.54
9187,Austria,2020-02-29,9.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149295,United Kingdom,2022-01-24,15992837.0,95317.0,154042.0,54.0,598.0,17210.0,429972989.0,0.0683,137474493.0,52252579.0,48224978.0,81.32
149296,United Kingdom,2022-01-25,16087344.0,94507.0,154485.0,443.0,575.0,16608.0,431317684.0,0.0686,137570759.0,52266515.0,48253766.0,81.32
149297,United Kingdom,2022-01-26,16189420.0,102076.0,154831.0,346.0,561.0,16514.0,432822845.0,0.0687,137671500.0,52281873.0,48284685.0,81.32
149298,United Kingdom,2022-01-27,16286017.0,96597.0,155169.0,338.0,549.0,16149.0,NaN,NaN,137770110.0,52297579.0,48314633.0,81.32


Replace NaNs with zero:

In [257]:
covid_europe = covid_europe.fillna(0)

In [258]:
covid_europe

,location,date,total_cases,new_cases,total_deaths,new_deaths,icu_patients,hosp_patients,total_tests,positive_rate,total_vaccinations,people_vaccinated,people_fully_vaccinated,life_expectancy
9183,Austria,2020-02-25,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,81.54
9184,Austria,2020-02-26,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,81.54
9185,Austria,2020-02-27,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,81.54
9186,Austria,2020-02-28,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,81.54
9187,Austria,2020-02-29,9.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,81.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149295,United Kingdom,2022-01-24,15992837.0,95317.0,154042.0,54.0,598.0,17210.0,429972989.0,0.0683,137474493.0,52252579.0,48224978.0,81.32
149296,United Kingdom,2022-01-25,16087344.0,94507.0,154485.0,443.0,575.0,16608.0,431317684.0,0.0686,137570759.0,52266515.0,48253766.0,81.32
149297,United Kingdom,2022-01-26,16189420.0,102076.0,154831.0,346.0,561.0,16514.0,432822845.0,0.0687,137671500.0,52281873.0,48284685.0,81.32
149298,United Kingdom,2022-01-27,16286017.0,96597.0,155169.0,338.0,549.0,16149.0,0.0,0.0000,137770110.0,52297579.0,48314633.0,81.32


## Country treatment

In [259]:
countries.head()

,gdp,sex_ratio,surface_area,life_expectancy_male,unemployment,imports,homicide_rate,currency,iso2,employment_services,...,name,pop_growth,region,pop_density,internet_users,gdp_per_capita,fertility,refugees,primary_school_enrollment_male,location
0,15059.0,103.7,28748.0,76.7,12.8,5908.0,2.3,"{'code': 'ALL', 'name': 'Lek'}",AL,43.7,...,Albania,-0.1,Southern Europe,105.0,71.8,5223.8,1.6,4.3,105.2,Albania
0,3238.0,102.3,468.0,0.0,0.0,1538.0,0.0,"{'code': 'EUR', 'name': 'Euro'}",AD,0.0,...,Andorra,-0.2,Southern Europe,164.2,91.6,42051.6,1.2,0.0,0.0,Andorra
0,455508.0,97.2,83871.0,78.9,4.8,176596.0,1.0,"{'code': 'EUR', 'name': 'Euro'}",AT,71.5,...,Austria,0.7,Western Europe,109.3,87.5,51230.3,1.5,163.7,103.2,Austria
0,59662.0,87.1,207600.0,69.3,4.6,39317.0,2.4,"{'code': 'BYN', 'name': 'Belarusian Ruble'}",BY,59.0,...,Belarus,0.0,Eastern Europe,46.6,79.1,6311.7,1.7,8.6,100.6,Belarus
0,543026.0,98.3,30528.0,79.0,5.7,426489.0,1.7,"{'code': 'EUR', 'name': 'Euro'}",BE,78.4,...,Belgium,0.5,Western Europe,382.7,88.7,47293.0,1.7,83.5,103.9,Belgium


In [260]:
columns_to_include = [
    "location"
    , "surface_area"
    , "population"
    , "employment_industry"
    , "internet_users"
    , "iso2"
    , "region"
]

In [261]:
countries.columns

Index(['gdp', 'sex_ratio', 'surface_area', 'life_expectancy_male',
       'unemployment', 'imports', 'homicide_rate', 'currency', 'iso2',
       'employment_services', 'employment_industry', 'urban_population_growth',
       'secondary_school_enrollment_female', 'employment_agriculture',
       'capital', 'co2_emissions', 'forested_area', 'tourists', 'exports',
       'life_expectancy_female', 'post_secondary_enrollment_female',
       'post_secondary_enrollment_male', 'primary_school_enrollment_female',
       'infant_mortality', 'gdp_growth', 'threatened_species', 'population',
       'urban_population', 'secondary_school_enrollment_male', 'name',
       'pop_growth', 'region', 'pop_density', 'internet_users',
       'gdp_per_capita', 'fertility', 'refugees',
       'primary_school_enrollment_male', 'location'],
      dtype='object')

In [262]:
countries = countries.loc[ : , columns_to_include ]

In [263]:
countries.head()

,location,surface_area,population,employment_industry,internet_users,iso2,region
0,Albania,28748.0,2878.0,20.2,71.8,AL,Southern Europe
0,Andorra,468.0,77.0,0.0,91.6,AD,Southern Europe
0,Austria,83871.0,9006.0,25.0,87.5,AT,Western Europe
0,Belarus,207600.0,9449.0,30.2,79.1,BY,Eastern Europe
0,Belgium,30528.0,11590.0,20.6,88.7,BE,Western Europe


In [264]:
covid_europe["location"] = covid_europe["location"].astype('str')
countries["location"] = countries["location"].astype('str')

## Exports

In [265]:
covid_europe = covid_europe.merge( countries , on="location"  , how="inner" )

In [266]:
covid_europe.shape

(17024, 20)

As a final test let's validate regions to export

In [270]:
covid_europe['region'].drop_duplicates()

0        Western Europe
1429     Eastern Europe
2121    Southern Europe
2825       Western Asia
4216    Northern Europe
Name: region, dtype: object

como podemos observar ciertos paises estan clasificados como parte de Asia, para no generar confusion por el momento los vamos a excluir.

In [273]:
covid_europe = covid_europe.loc[ ~(covid_europe['region']=='Western Asia') , : ]

In [274]:
covid_europe.to_csv( "../data_source/covid_europe.csv" , index_label=False )